<a href="https://colab.research.google.com/github/mushir2004/train-and-test/blob/main/Trye_Ageing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install pandas scikit-learn openpyxl


In [11]:
import pandas as pd

# Load the Excel file
file_path = '/content/AGING TYRE RFID_2.xlsx'
data = pd.read_excel(file_path, engine='openpyxl')

# Display the first few rows of the dataset
print(data.head())


            Tire Size: Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0                  NaN        NaN        NaN       Temp      Dist.       Rad.   
1                 Date       Time       Step       (ºC)       (km)       (cm)   
2  2022-03-07 00:00:00   13:09:01          1       30.1          0       49.9   
3  2022-03-07 00:00:00   13:10:01          1         29          0       49.9   
4  2022-03-07 00:00:00   13:11:01          1       29.1          0       49.9   

  Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ...                Unnamed: 18  \
0        Spd   Req. Ld.    Act. Ld  Req. Inf.  ...                        NaN   
1      (kph)       (kg)       (kg)      (psi)  ...               Explanations   
2          0       2343    2337.05          0  ...  Resuming or Starting Step   
3         72       2343    2343.24          0  ...                        NaN   
4         72       2343    2342.71          0  ...                        NaN   

  Unnamed: 19 Unnamed: 20 

In [17]:
# Fill missing values with a placeholder
data['Unnamed: 5'] = data['Unnamed: 5'].fillna('Unnamed: 5')


In [18]:
# Convert all values in the 'Target' column to strings
data['Unnamed: 5'] = data['Unnamed: 5'].astype(str)


In [19]:
# Check the data types in the column
print(data['Unnamed: 5'].unique())


['Rad.' '(cm)' '49.9' '50' '50.1' '50.2' '49.6' '49.7' '49.2' '49' '48.7'
 '48.8' '48.5' '48.6' '48.3' '46.2' '44.7' '43.2' '41.8' '40.5' '40']


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Example: Encode categorical variables if any
label_encoder = LabelEncoder()

# Assuming 'Target' is the column you want to predict
data['Unnamed: 5'] = label_encoder.fit_transform(data['Unnamed: 5'])

# Separate features and target variable
X = data.drop(columns=['Unnamed: 4'])  # Features
y = data['Unnamed: 5']  # Target variable

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
# Convert the 'Unnamed: 1' column to datetime format
X_train['Unnamed: 1'] = pd.to_datetime(X_train['Unnamed: 1'], errors='coerce')

# Now you can extract day, month, year, etc.
X_train['Day'] = X_train['Unnamed: 1'].dt.day
X_train['Month'] = X_train['Unnamed: 1'].dt.month
X_train['Year'] = X_train['Unnamed: 1'].dt.year

# Drop the original datetime column if it's no longer needed
X_train = X_train.drop(columns=['Unnamed: 1'])


In [28]:
# Check for non-numeric columns
print(X_train.dtypes)

# If there are any non-numeric columns, handle them accordingly (either drop or encode)


Tire Size:      object
Unnamed: 2      object
Unnamed: 3      object
Unnamed: 5       int64
Unnamed: 6      object
Unnamed: 7      object
Unnamed: 8      object
Unnamed: 9      object
Unnamed: 10     object
Unnamed: 11     object
Unnamed: 12     object
Unnamed: 13     object
Unnamed: 14     object
Unnamed: 15     object
Unnamed: 16     object
Unnamed: 17     object
Unnamed: 18     object
Unnamed: 19    float64
Unnamed: 20    float64
Unnamed: 21    float64
Unnamed: 22    float64
Unnamed: 23    float64
Unnamed: 24    float64
Unnamed: 25    float64
Unnamed: 26    float64
Unnamed: 27    float64
Day            float64
Month          float64
Year           float64
dtype: object


In [30]:
# Check the data types of all columns in X_train
print(X_train.dtypes)

# Check for non-numeric columns
non_numeric_columns = X_train.select_dtypes(exclude=['int', 'float']).columns
print("Non-numeric columns: ", non_numeric_columns)


Tire Size:      object
Unnamed: 2      object
Unnamed: 3      object
Unnamed: 5       int64
Unnamed: 6      object
Unnamed: 7      object
Unnamed: 8      object
Unnamed: 9      object
Unnamed: 10     object
Unnamed: 11     object
Unnamed: 12     object
Unnamed: 13     object
Unnamed: 14     object
Unnamed: 15     object
Unnamed: 16     object
Unnamed: 17     object
Unnamed: 18     object
Unnamed: 19    float64
Unnamed: 20    float64
Unnamed: 21    float64
Unnamed: 22    float64
Unnamed: 23    float64
Unnamed: 24    float64
Unnamed: 25    float64
Unnamed: 26    float64
Unnamed: 27    float64
Day            float64
Month          float64
Year           float64
dtype: object
Non-numeric columns:  Index(['Tire Size:', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')


In [31]:
# Use one-hot encoding to convert categorical columns into numeric
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)


In [34]:
model.fit(X_train, y_train)


DecisionTreeClassifier()

In [36]:
# Combine train and test to ensure consistent one-hot encoding
combined = pd.concat([X_train, X_test], keys=['train', 'test'])

# One-hot encode the combined data
combined_encoded = pd.get_dummies(combined, drop_first=True)

# Split the combined data back into X_train and X_test
X_train = combined_encoded.xs('train')
X_test = combined_encoded.xs('test')


In [37]:
# Align columns in X_test to match X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


In [38]:
# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the classifier
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 41.99%
